In [2]:
import os
import sys
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
train = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'train수작업_meta.xlsx'))

In [79]:
test_sale = pd.read_excel(os.path.join('..', '..', '0.Data', '02_평가데이터', '2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx'), skiprows = 1)

In [17]:
test = pd.read_excel(os.path.join('..', '..', '0.Data', '02_평가데이터', 'test수작업_meta.xlsx'))

In [34]:
item = pd.read_csv(os.path.join('..', '..', '0.Data', '01_제공데이터', 'item_meta.csv'), encoding = 'cp949')

In [47]:
item = item.merge(train.drop_duplicates('NEW상품명')[['NEW상품명', '마더코드', '상품코드']], on = 'NEW상품명', how = 'left')

In [42]:
test['NEW상품명'] = test['NEW상품명'].apply(lambda x : x.strip())

In [52]:
temp = test.merge(item, on = 'NEW상품명', how = 'left')

In [62]:
item

,NEW상품코드,NEW상품명,상품군,NEW_최고판매단가,NEW_최저판매단가,NEW_평균판매단가,NEW_중간판매단가,NEW_최고-최저,NEW_분산,NEW_표준편차,전체_가격대,상품군_가격대,마더코드,상품코드
0,0,가이거 남성 블랙 에디션 다이아몬드 워치,잡화,178000,178000,178000.0,178000,0,0.0,0.000000,고가,1.0,100292,200944
1,1,가이거 무빙스타 다이아몬드 워치,잡화,129000,119000,124000.0,124000,10000,50000000.0,7071.067812,고가,1.0,100293,200948
2,2,가이거 무빙스타 다이아몬드 워치 + 보석함,잡화,119000,99000,109000.0,109000,20000,200000000.0,14142.135624,고가,1.0,100293,200946
3,3,가이거 여성 블랙 에디션 다이아몬드 워치,잡화,178000,178000,178000.0,178000,0,0.0,0.000000,고가,1.0,100292,200945
4,4,가이거 제니스시계 주얼리세트,잡화,139000,129000,134000.0,134000,10000,50000000.0,7071.067812,고가,1.0,100293,200949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,1350,AAB의 든든한 콩국수,농수축,39900,39900,39900.0,39900,0,NaN,NaN,저가,0.0,100009,200013
1351,1351,AAB의 소곱창 전골 800g 8팩,농수축,55900,55900,55900.0,55900,0,NaN,NaN,저가,2.0,100010,200014
1352,1352,AAE LA 갈비 7팩,농수축,69900,69900,69900.0,69900,0,NaN,NaN,중저가,2.0,100022,200033
1353,1353,IH 옛 가마솥 세트,주방,62000,62000,62000.0,62000,0,NaN,NaN,중저가,0.0,100724,202115


In [66]:
sale = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', '2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx'), skiprows = 1)

In [72]:
sale.shape

(38309, 8)

In [64]:
meta = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'train수작업_meta.xlsx'))

In [68]:
df = sale.merge(meta[['상품코드', 'NEW상품명', '브랜드', '결제방법', '상품명다시', '단위', '모델명', '성별', 'NS카테고리']], on = '상품코드', how = 'left')

In [89]:
list(meta.columns)

['상품코드',
 '상품명',
 '상품군',
 '판매단가',
 '결제방법',
 '상품명다시',
 '단위',
 '브랜드',
 '옵션',
 '모델명',
 '성별',
 'NEW상품명',
 'NEW상품코드',
 '마더코드',
 'NS카테고리']

In [78]:
test.columns

Index(['마더코드', '상품코드', '상품명', '상품군', '결제방법', '브랜드', '상품명다시', '단위', '옵션', '종류',
       '모델명', 'main', 'NEW상품명'],
      dtype='object')

In [82]:
test = test.merge(test_sale[['상품코드', '판매단가']].drop_duplicates(), on = '상품코드', how = 'left')

In [84]:
test = test.rename(columns = {'main' : 'NS카테고리'})

In [93]:
test['성별'] = test['NEW상품명'].apply(lambda x : 1 if '남성' in x else (2 if '여성' in x else 0))

In [97]:
test = test[['상품코드',
 '상품명',
 '상품군',
 '판매단가',
 '결제방법',
 '상품명다시',
 '단위',
 '브랜드',
 '옵션',
 '모델명',
 '성별',
 'NEW상품명',
 '마더코드',
 'NS카테고리']]

In [100]:
item

,NEW상품코드,NEW상품명,상품군,NEW_최고판매단가,NEW_최저판매단가,NEW_평균판매단가,NEW_중간판매단가,NEW_최고-최저,NEW_분산,NEW_표준편차,전체_가격대,상품군_가격대,마더코드,상품코드
0,0,가이거 남성 블랙 에디션 다이아몬드 워치,잡화,178000,178000,178000.0,178000,0,0.0,0.000000,고가,1.0,100292,200944
1,1,가이거 무빙스타 다이아몬드 워치,잡화,129000,119000,124000.0,124000,10000,50000000.0,7071.067812,고가,1.0,100293,200948
2,2,가이거 무빙스타 다이아몬드 워치 + 보석함,잡화,119000,99000,109000.0,109000,20000,200000000.0,14142.135624,고가,1.0,100293,200946
3,3,가이거 여성 블랙 에디션 다이아몬드 워치,잡화,178000,178000,178000.0,178000,0,0.0,0.000000,고가,1.0,100292,200945
4,4,가이거 제니스시계 주얼리세트,잡화,139000,129000,134000.0,134000,10000,50000000.0,7071.067812,고가,1.0,100293,200949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,1350,AAB의 든든한 콩국수,농수축,39900,39900,39900.0,39900,0,NaN,NaN,저가,0.0,100009,200013
1351,1351,AAB의 소곱창 전골 800g 8팩,농수축,55900,55900,55900.0,55900,0,NaN,NaN,저가,2.0,100010,200014
1352,1352,AAE LA 갈비 7팩,농수축,69900,69900,69900.0,69900,0,NaN,NaN,중저가,2.0,100022,200033
1353,1353,IH 옛 가마솥 세트,주방,62000,62000,62000.0,62000,0,NaN,NaN,중저가,0.0,100724,202115


In [98]:
test

,상품코드,상품명,상품군,판매단가,결제방법,상품명다시,단위,브랜드,옵션,모델명,성별,NEW상품명,마더코드,NS카테고리
0,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,남성 반팔 셔츠,4종,잭필드,NaN,NaN,1,잭필드 남성 반팔 셔츠 4종,100650,남성의류
1,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,0.0,여성 쿨 레이시 란쥬 쉐이퍼 & 팬티,NaN,쿠미투니카,NaN,NaN,2,쿠미투니카 여성 쿨 레이시 란쥬 쉐이퍼 & 팬티,100445,언더웨어
2,201247,바비리스 퍼펙트 볼륨스타일러,이미용,59000,NaN,퍼펙트 볼륨 스타일러,NaN,바비리스,NaN,NaN,0,바비리스 퍼펙트 볼륨 스타일러,100381,헤어/바디
3,201956,램프쿡 자동회전냄비,주방,109000,0.0,자동 회전 냄비,NaN,램프쿡,NaN,NaN,0,램프쿡 자동 회전 냄비,100638,주방용품/식기
4,201091,벨레즈온 심리스 원피스 4종 패키지,속옷,59900,0.0,여성 심리스 원피스 패키지,4종,벨레즈온,NaN,NaN,2,벨레즈온 여성 심리스 원피스 패키지 4종,100348,언더웨어
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,202427,QISS 프리미어 아침애끌리다 마시는한끼 21통,농수축,56900,0.0,프리미어 아침에 끌리다 마시는한끼,21통,QISS,NaN,NaN,0,QISS 프리미어 아침에 끌리다 마시는한끼 21통,100825,건강기능
413,202428,QISS 프리미어 아침애끌리다 마시는한끼 42통,농수축,76900,0.0,프리미어 아침에 끌리다 마시는한끼,42통,QISS,NaN,NaN,0,QISS 프리미어 아침에 끌리다 마시는한끼 42통,100825,건강기능
414,200006,W클라우드 남성 데일리 트레킹화 3종,잡화,49800,0.0,남성 데일리 트레킹화,3종,W클라우드,NaN,NaN,1,W클라우드 남성 데일리 트레킹화 3종,100005,스포츠의류/슈즈/잡화
415,200007,W클라우드 여성 데일리 트레킹화 3종,잡화,49800,0.0,여성 데일리 트레킹화,3종,W클라우드,NaN,NaN,2,W클라우드 여성 데일리 트레킹화 3종,100005,스포츠의류/슈즈/잡화


In [ ]:
data = pd.concat([])

In [102]:
item

,NEW상품코드,NEW상품명,상품군,NEW_최고판매단가,NEW_최저판매단가,NEW_평균판매단가,NEW_중간판매단가,NEW_최고-최저,NEW_분산,NEW_표준편차,전체_가격대,상품군_가격대,마더코드,상품코드
0,0,가이거 남성 블랙 에디션 다이아몬드 워치,잡화,178000,178000,178000.0,178000,0,0.0,0.000000,고가,1.0,100292,200944
1,1,가이거 무빙스타 다이아몬드 워치,잡화,129000,119000,124000.0,124000,10000,50000000.0,7071.067812,고가,1.0,100293,200948
2,2,가이거 무빙스타 다이아몬드 워치 + 보석함,잡화,119000,99000,109000.0,109000,20000,200000000.0,14142.135624,고가,1.0,100293,200946
3,3,가이거 여성 블랙 에디션 다이아몬드 워치,잡화,178000,178000,178000.0,178000,0,0.0,0.000000,고가,1.0,100292,200945
4,4,가이거 제니스시계 주얼리세트,잡화,139000,129000,134000.0,134000,10000,50000000.0,7071.067812,고가,1.0,100293,200949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,1350,AAB의 든든한 콩국수,농수축,39900,39900,39900.0,39900,0,NaN,NaN,저가,0.0,100009,200013
1351,1351,AAB의 소곱창 전골 800g 8팩,농수축,55900,55900,55900.0,55900,0,NaN,NaN,저가,2.0,100010,200014
1352,1352,AAE LA 갈비 7팩,농수축,69900,69900,69900.0,69900,0,NaN,NaN,중저가,2.0,100022,200033
1353,1353,IH 옛 가마솥 세트,주방,62000,62000,62000.0,62000,0,NaN,NaN,중저가,0.0,100724,202115


In [106]:
test_item = test.drop_duplicates('NEW상품명')[['NEW상품명', '상품군', '마더코드', '상품코드']]

In [107]:
test_item

,NEW상품명,상품군,마더코드,상품코드
0,잭필드 남성 반팔 셔츠 4종,의류,100650,201971
1,쿠미투니카 여성 쿨 레이시 란쥬 쉐이퍼 & 팬티,속옷,100445,202278
2,바비리스 퍼펙트 볼륨 스타일러,이미용,100381,201247
3,램프쿡 자동 회전 냄비,주방,100638,201956
4,벨레즈온 여성 심리스 원피스 패키지 4종,속옷,100348,201091
...,...,...,...,...
412,QISS 프리미어 아침에 끌리다 마시는한끼 21통,농수축,100825,202427
413,QISS 프리미어 아침에 끌리다 마시는한끼 42통,농수축,100825,202428
414,W클라우드 남성 데일리 트레킹화 3종,잡화,100005,200006
415,W클라우드 여성 데일리 트레킹화 3종,잡화,100005,200007


In [110]:
item = pd.concat([item, test_item], axis = 0)

In [118]:
item.drop('NEW상품코드', axis = 1).reset_index(drop = True).reset_index().rename(columns = {'index' : 'NEW상품코드'})

,NEW상품코드,NEW상품명,상품군,NEW_최고판매단가,NEW_최저판매단가,NEW_평균판매단가,NEW_중간판매단가,NEW_최고-최저,NEW_분산,NEW_표준편차,전체_가격대,상품군_가격대,마더코드,상품코드
0,0,가이거 남성 블랙 에디션 다이아몬드 워치,잡화,178000.0,178000.0,178000.0,178000.0,0.0,0.0,0.000000,고가,1.0,100292,200944
1,1,가이거 무빙스타 다이아몬드 워치,잡화,129000.0,119000.0,124000.0,124000.0,10000.0,50000000.0,7071.067812,고가,1.0,100293,200948
2,2,가이거 무빙스타 다이아몬드 워치 + 보석함,잡화,119000.0,99000.0,109000.0,109000.0,20000.0,200000000.0,14142.135624,고가,1.0,100293,200946
3,3,가이거 여성 블랙 에디션 다이아몬드 워치,잡화,178000.0,178000.0,178000.0,178000.0,0.0,0.0,0.000000,고가,1.0,100292,200945
4,4,가이거 제니스시계 주얼리세트,잡화,139000.0,129000.0,134000.0,134000.0,10000.0,50000000.0,7071.067812,고가,1.0,100293,200949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,1662,QISS 프리미어 아침에 끌리다 마시는한끼 21통,농수축,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100825,202427
1663,1663,QISS 프리미어 아침에 끌리다 마시는한끼 42통,농수축,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100825,202428
1664,1664,W클라우드 남성 데일리 트레킹화 3종,잡화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100005,200006
1665,1665,W클라우드 여성 데일리 트레킹화 3종,잡화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100005,200007


In [147]:
meta_train = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'train수작업_meta.xlsx'))
meta_test = pd.read_excel(os.path.join('..', '..', '0.Data', '02_평가데이터', 'test수작업_meta.xlsx'))

In [149]:
meta = pd.concat([meta_train, meta_test], axis = 0)

In [150]:
meta[['NEW상품명', '상품군']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index' : 'NEW상품코드'})

,NEW상품코드,NEW상품명,상품군
0,0,가이거 남성 블랙 에디션 다이아몬드 워치,잡화
1,1,가이거 무빙스타 다이아몬드 워치,잡화
2,2,가이거 무빙스타 다이아몬드 워치 + 보석함,잡화
3,3,가이거 여성 블랙 에디션 다이아몬드 워치,잡화
4,4,가이거 제니스시계 주얼리세트,잡화
...,...,...,...
1620,1620,QISS 프리미어 아침에 끌리다 마시는한끼 21통,농수축
1621,1621,QISS 프리미어 아침에 끌리다 마시는한끼 42통,농수축
1622,1622,W클라우드 남성 데일리 트레킹화 3종,잡화
1623,1623,W클라우드 여성 데일리 트레킹화 3종,잡화


In [57]:
temp.loc[temp['마더코드_y'].notnull(), ['NEW상품명', '마더코드_x', '마더코드_y']].head(60)

,NEW상품명,마더코드_x,마더코드_y
3,램프쿡 자동 회전 냄비,100638,100638.0
14,LG 통돌이 세탁기,100150,100150.0
16,노비타 스마트 비데,100501,100501.0
23,올바로 문어발 샤워기,100320,100320.0
24,올바로 문어발 샤워기,100320,100320.0
29,더블모 어성초 샴푸,100262,100262.0
30,마르엘라로사티 린넨 베스트 세트,100353,100353.0
40,캐리어 벽걸이 에어컨 7형,100068,100067.0
50,LG 매직스페이스 냉장고,100155,100155.0
51,LG 매직스페이스 냉장고,100155,100155.0
